In [1]:
# HOMEWORK 4 - create a database, parsing some site
#              and put the data to the database
#

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import json
import sqlite3

In [3]:
# For this task I parsed website 'https://melonpanda.com' of the 
# internet cosmetics shop. I'm creating 2 tables:
#     1. brands   - list of brands (id, name)
#     2. products - list of products (id, brand_id, name,
#                                     price, sold_out_flag)
#
# Analysis of the website's pages suggest that the list of brands and 
# the list of products are a JavaScript variables. And the values of 
# these variables are in JSON format. Therefore I wrote a function to 
# get JSON data by URI and variable names.

In [4]:
# get JSON data by URI and JS variable name.
def get_json_value_by_js_variable_name(uri, var_name):

    # get requested page
    rep = requests.get(uri)

    # find script tags by using BeautifulSoup
    soup = BeautifulSoup(rep.text, 'html.parser')
    scripts = soup.find_all("script")

    # find variable in scripts,
    # JS the code has view: ... requested.var.name = [value]; ...
    p = re.compile(f'{var_name} = \[.*?\];')
    matches = p.findall(str(scripts))
    if not matches:
        print('Err:  could not find variable={var_name}.')
        return None

    # extract variable value - skip variable name and list symbol ';'
    json_str = matches[0][len(var_name) + 3 : -1]
    var_value = json.loads(json_str)

    return var_value

In [6]:
# Now create DB and tables. Also we will clean them befor filling.
conn = sqlite3.connect('hw4.db')

In [11]:
# create brands table
query = '''
            CREATE TABLE IF NOT EXISTS brands(
                    brand_id INTEGER PRIMARY KEY,
                    name TEXT
            )
        '''
conn.execute(query)      

# create product table
query = '''
            CREATE TABLE IF NOT EXISTS products(
                    product_id INTEGER PRIMARY KEY,
                    brand_id,
                    name TEXT,
                    price FLOAT,
                    sold_out BOOL
            )
        '''
conn.execute(query)

# clear tables before filling
conn.execute('DELETE FROM brands')
conn.execute('DELETE FROM products')
conn.commit()

In [12]:
# and now we are parsing site and filling our 2 tables

# get list of brands
uri = 'https://melonpanda.com/ru/catalog/brands'
var = 'window.site.binding.brands'
brands = get_json_value_by_js_variable_name(uri, var)

# go through the brands
brand_id = 0
for brand in brands:

    brand_id += 1

    # since this is a learning project, collect info only about first 5 brands
    if brand_id > 5:
        break

    print(brand_id, brand['name'], brand['url'])

    # get list of products
    var = 'window.site.binding.products'
    products = get_json_value_by_js_variable_name(brand['url'], var)

    # go the products and prepare data for DB
    products_prepared = []
    for prod in products:

        #for k in prod:
        #    print(k, ':', prod[k])

        name = prod['name']
        prices = prod['prices']
        sold_out = prod['sold_out']

        # the field 'prices' is another one dict, get price in RUB
        price = float('nan')
        for pr in prices:
            if pr['currency'] == 'rub':
                price = pr['value']

        # store product in list
        products_prepared.append([brand_id, name, price, sold_out])


    # and now add data to DB

    # add brand to brand table
    query = f'''
                INSERT INTO
                brands(brand_id, name)
                VALUES({brand_id}, '{name}')
            '''
    conn.execute(query)

    # add products to product table
    query = '''
                INSERT INTO
                products(brand_id, name, price, sold_out)
                VALUES(?, ?, ?, ?)
            '''
    conn.executemany(query, products_prepared)

    # commit the result
    conn.commit()

1 ADDICTION https://melonpanda.com/ru/catalog/brand/addiction
2 ALOVIVI https://melonpanda.com/ru/catalog/brand/alovivi
3 APAGARD https://melonpanda.com/ru/catalog/brand/apagard
4 AROUGE https://melonpanda.com/ru/catalog/brand/arouge
5 Bb Laboratories https://melonpanda.com/ru/catalog/brand/bb-laboratories


In [13]:
# example of using DB

# at first get all products with price > 5000
query = '''
            SELECT product_id, brand_id, name, price, sold_out
            FROM products
            WHERE price > 5000
        '''
res = conn.execute(query).fetchall()

print('Original result of SELECT:')
for i in res:
    print(i)

# update values - set zero price if product is sold out
query = '''
            UPDATE products
            SET price = 0
            WHERE sold_out == True
        '''
conn.execute(query)

# and make the same SELECT request once again to see the result
query = '''
            SELECT product_id, brand_id, name, price, sold_out
            FROM products
            WHERE price > 5000
        '''
res = conn.execute(query).fetchall()

print('\nResult of SELECT after UPDATE:')
for i in res:
    print(i)
    
# commit the changes to DB
conn.commit()    

Original result of SELECT:
(5, 1, 'The Glow Foundation — тональный крем с влажным эффектом', 5410.51, 0)
(11, 1, 'Chelsea Nail Collection — рождественский набор', 6377.76, 1)
(13, 1, 'Ready To Wear Eye Palette 03', 7584.21, 1)
(14, 1, 'Ready To Wear Eye Palette 02', 7584.21, 1)
(15, 1, 'Ready To Wear Eye Palette 01', 7584.21, 1)
(39, 5, 'Emollient Lift Lotion — лосьон для лица', 6350.23, 0)
(40, 5, 'Placenta Extract — жидкий экстракт плаценты, 30 мл.', 10716.61, 0)
(41, 5, 'Hyalurone Elastin Collagen Extract — антивозрастной коктейль, 30 мл.', 8573.29, 0)
(42, 5, 'Hyalurone Elastin Collagen Extract — антивозрастной коктейль, 50 мл.', 12149.93, 0)
(44, 5, 'Emollient Lift Lotion Set — набор с миниатюрами', 6350.23, 1)
(45, 5, 'Hyaluron Elastin Collagen — комплекс нутриентов', 8504.95, 1)
(46, 5, 'Hyaluron Concentrate Water — гиалуроновый лосьон', 8903.62, 1)
(47, 5, 'Emollient Lift Cream — питательный лифтинг-крем', 12529.61, 1)
(48, 5, 'Clear Line EX Stick — стик против мимических морщи

In [14]:
# finaly close the DB
conn.close()